In [21]:
#    This file is part of DEAP.
#
#    DEAP is free software: you can redistribute it and/or modify
#    it under the terms of the GNU Lesser General Public License as
#    published by the Free Software Foundation, either version 3 of
#    the License, or (at your option) any later version.
#
#    DEAP is distributed in the hope that it will be useful,
#    but WITHOUT ANY WARRANTY; without even the implied warranty of
#    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the
#    GNU Lesser General Public License for more details.
#
#    You should have received a copy of the GNU Lesser General Public
#    License along with DEAP. If not, see <http://www.gnu.org/licenses/>.

import random

import numpy

import algorithms
from deap import base
from deap import creator
from deap import tools

In [22]:
IND_INIT_SIZE = 5
MAX_ITEM = 50
MAX_WEIGHT = 50
NBR_ITEMS = 20

# To assure reproductibility, the RNG seed is set prior to the items
# dict initialization. It is also seeded in main().
random.seed(64)

In [23]:
# Create the item dictionary: item name is an integer, and value is 
# a (weight, value) 2-uple.
items = {}
# Create random items and store them in the items' dictionary.
for i in range(NBR_ITEMS):
    items[i] = (random.randint(1, 10), random.uniform(0, 100))

In [24]:
creator.create("Fitness", base.Fitness, weights=(-1.0, 1.0))
creator.create("Individual", set, fitness=creator.Fitness)

toolbox = base.Toolbox()

# Attribute generator
toolbox.register("attr_item", random.randrange, NBR_ITEMS)

# Structure initializers
toolbox.register("individual", tools.initRepeat, creator.Individual, 
    toolbox.attr_item, IND_INIT_SIZE)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [25]:
def evalKnapsack(individual):
    weight = 0.0
    value = 0.0
    for item in individual:
        weight += items[item][0]
        value += items[item][1]
    if len(individual) > MAX_ITEM or weight > MAX_WEIGHT:
        return 10000, 0             # Ensure overweighted bags are dominated
    return weight, value

In [26]:
def cxSet(ind1, ind2):
    """Apply a crossover operation on input sets. The first child is the
    intersection of the two sets, the second child is the difference of the
    two sets.
    """
    temp = set(ind1)                # Used in order to keep type
    ind1 &= ind2                    # Intersection (inplace)
    ind2 ^= temp                    # Symmetric Difference (inplace)
    return ind1, ind2

In [27]:
def mutSet(individual):
    """Mutation that pops or add an element."""
    if random.random() < 0.5:
        if len(individual) > 0:     # We cannot pop from an empty set
            individual.remove(random.choice(sorted(tuple(individual))))
    else:
        individual.add(random.randrange(NBR_ITEMS))
    return individual,

toolbox.register("evaluate", evalKnapsack)
toolbox.register("mate", cxSet)
toolbox.register("mutate", mutSet)
toolbox.register("select", tools.selNSGA2)

In [28]:
def main():
    random.seed(64)
    NGEN = 50
    MU = 50
    LAMBDA = 100
    CXPB = 0.7
    MUTPB = 0.2
    
    pop = toolbox.population(n=MU)
    hof = tools.ParetoFront()
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean, axis=0)
    stats.register("std", numpy.std, axis=0)
    stats.register("min", numpy.min, axis=0)
    stats.register("max", numpy.max, axis=0)
    
    algorithms.eaMuPlusLambda(pop, toolbox, MU, LAMBDA, CXPB, MUTPB, NGEN, stats,
                              halloffame=hof)
    
    return pop, stats, hof

In [29]:
if __name__ == "__main__":
    main()

gen	nevals	avg                          	std                        	min                        	max                          
0  	50    	[  22.78        210.00877715]	[  5.88316241  71.09309649]	[ 10.          49.69958685]	[  38.          345.35491309]
0  	87    	[   9.96        145.20790666]	[  10.61312395  139.1868852 ]	[ 0.  0.]                  	[  45.          414.48478381]
t: 0 | D: 1.844638601395483 M: 1.844638601395483 S: 1.844638601395483
1  	91    	[  3.26        61.20087478]  	[   7.44797959  125.53892809]	[ 0.  0.]                  	[  28.          414.48478381]
t: 1 | D: 1.8915454479293066 M: 1.8915454479293066 S: 1.8915454479293066
2  	88    	[  4.6         84.51641114]  	[   8.57438044  140.51459866]	[ 0.  0.]                  	[  28.          414.48478381]
t: 2 | D: 0.11864836876212398 M: 1.9274 S: 1.6399
3  	92    	[  2.4         52.24310488]  	[   5.82065288  108.88598622]	[ 0.  0.]                  	[  28.          414.48478381]
t: 3 | D: 0.3776377286341025 M: 1.410

Matching M ->  0.1877
against ->  [0.3067, 0.3009, 0.2926, 0.2829, 0.2738, 0.2652, 0.2572, 0.2513, 0.2441, 0.2373, 0.2304, 0.2274, 0.2216, 0.2174, 0.2177, 0.214, 0.2164, 0.2114, 0.2093, 0.2059, 0.2014, 0.1972, 0.2028, 0.1984, 0.1945, 0.1918, 0.1953, 0.1916, 0.1912]
 Testing termination:
Matching S ->  0.1416
against ->  [0.2522, 0.244, 0.2373, 0.232, 0.2268, 0.2219, 0.2171, 0.2117, 0.2074, 0.2031, 0.1993, 0.1943, 0.1905, 0.1864, 0.1819, 0.1781, 0.1742, 0.1713, 0.1676, 0.1645, 0.1619, 0.1593, 0.1576, 0.1553, 0.153, 0.1504, 0.1482, 0.1461, 0.1435]
t: 56 | D: -0.002035799239553951 M: 0.1877 S: 0.1416
57 	91    	[  12.1         321.02585673]	[  12.54153101  240.86626983]	[ 0.  0.]                  	[  47.          719.71544879]
 Testing termination:
Matching M ->  0.1855
against ->  [0.3009, 0.2926, 0.2829, 0.2738, 0.2652, 0.2572, 0.2513, 0.2441, 0.2373, 0.2304, 0.2274, 0.2216, 0.2174, 0.2177, 0.214, 0.2164, 0.2114, 0.2093, 0.2059, 0.2014, 0.1972, 0.2028, 0.1984, 0.1945, 0.1918, 0.1953, 0.

 Testing termination:
Matching S ->  0.0821
against ->  [0.1069, 0.1058, 0.1047, 0.1035, 0.1032, 0.1022, 0.1012, 0.1002, 0.0993, 0.0983, 0.0974, 0.0965, 0.0956, 0.0947, 0.0938, 0.0928, 0.092, 0.0911, 0.0903, 0.0895, 0.0888, 0.0879, 0.0871, 0.0863, 0.0856, 0.0849, 0.0842, 0.0834, 0.0827]
t: 108 | D: 0.00022768942683179874 M: 0.1077 S: 0.0821
109	87    	[  21.04        488.67727567]	[  14.29120009  188.71223329]	[ 0.  0.]                  	[  50.          752.27685279]
 Testing termination:
Matching M ->  0.1067
against ->  [0.134, 0.1323, 0.1318, 0.1353, 0.1336, 0.132, 0.1305, 0.1289, 0.1274, 0.1259, 0.1246, 0.1232, 0.1218, 0.1209, 0.1202, 0.1189, 0.1182, 0.1169, 0.1157, 0.1146, 0.1143, 0.1132, 0.1135, 0.1124, 0.1114, 0.1103, 0.1098, 0.1087, 0.1077]
 Testing termination:
Matching S ->  0.0814
against ->  [0.1058, 0.1047, 0.1035, 0.1032, 0.1022, 0.1012, 0.1002, 0.0993, 0.0983, 0.0974, 0.0965, 0.0956, 0.0947, 0.0938, 0.0928, 0.092, 0.0911, 0.0903, 0.0895, 0.0888, 0.0879, 0.0871, 0.0863, 0

t: 160 | D: -0.02340859940093665 M: 0.0767 S: 0.0581
161	91    	[  20.16        473.54342787]	[  14.43517925  198.62079392]	[ 0.  0.]                  	[  50.          754.83691157]
 Testing termination:
Matching M ->  0.0762
against ->  [0.0924, 0.0917, 0.091, 0.0903, 0.0896, 0.089, 0.0883, 0.0877, 0.087, 0.0868, 0.0862, 0.0856, 0.085, 0.0844, 0.0834, 0.0828, 0.0823, 0.0817, 0.0812, 0.0806, 0.0801, 0.0794, 0.0788, 0.0789, 0.0784, 0.0783, 0.0778, 0.0773, 0.0767]
 Testing termination:
Matching S ->  0.0578
against ->  [0.0689, 0.0685, 0.068, 0.0676, 0.0671, 0.0667, 0.0663, 0.0659, 0.0654, 0.065, 0.0646, 0.0642, 0.0638, 0.0634, 0.0631, 0.0627, 0.0624, 0.062, 0.0616, 0.0612, 0.0609, 0.0606, 0.0602, 0.0598, 0.0595, 0.0591, 0.0588, 0.0584, 0.0581]
t: 161 | D: -0.010332144669305345 M: 0.0762 S: 0.0578
162	92    	[  21.54        491.48870691]	[  14.32788889  195.67350214]	[ 0.  0.]                  	[  50.          754.83691157]
 Testing termination:
Matching M ->  0.0757
against ->  [0.0917,

 Testing termination:
Matching M ->  0.0572
against ->  [0.0659, 0.0655, 0.0651, 0.0648, 0.0644, 0.0641, 0.0637, 0.0634, 0.0631, 0.0628, 0.0624, 0.0621, 0.0618, 0.0614, 0.0611, 0.0608, 0.0604, 0.0601, 0.0598, 0.0595, 0.0593, 0.059, 0.0587, 0.0584, 0.0581, 0.0583, 0.058, 0.058, 0.0575]
 Testing termination:
Matching S ->  0.0442
against ->  [0.0503, 0.0501, 0.0498, 0.0496, 0.0494, 0.0491, 0.0489, 0.0487, 0.0485, 0.0482, 0.048, 0.0478, 0.0476, 0.0474, 0.0471, 0.0469, 0.0467, 0.0465, 0.0463, 0.0461, 0.0459, 0.0457, 0.0455, 0.0453, 0.0451, 0.0449, 0.0447, 0.0445, 0.0443]
t: 217 | D: 0.0007227809323174447 M: 0.0572 S: 0.0442
218	89    	[  22.3         505.20127253]	[  14.71767645  188.05359337]	[ 0.  0.]                  	[  50.          754.83691157]
 Testing termination:
Matching M ->  0.0569
against ->  [0.0655, 0.0651, 0.0648, 0.0644, 0.0641, 0.0637, 0.0634, 0.0631, 0.0628, 0.0624, 0.0621, 0.0618, 0.0614, 0.0611, 0.0608, 0.0604, 0.0601, 0.0598, 0.0595, 0.0593, 0.059, 0.0587, 0.0584, 0.0

...

against ->  [0.017, 0.017, 0.017, 0.017, 0.017, 0.017, 0.017, 0.017, 0.017, 0.017, 0.017, 0.017, 0.017, 0.0169, 0.0169, 0.0169, 0.0169, 0.0169, 0.0169, 0.0169, 0.0169, 0.0169, 0.0169, 0.0169, 0.0169, 0.0169, 0.0169, 0.0169, 0.0169]
 Testing termination:
Matching S ->  0.0053
against ->  [0.0053, 0.0053, 0.0053, 0.0053, 0.0053, 0.0053, 0.0053, 0.0053, 0.0053, 0.0053, 0.0053, 0.0053, 0.0053, 0.0053, 0.0053, 0.0053, 0.0053, 0.0053, 0.0053, 0.0053, 0.0053, 0.0053, 0.0053, 0.0053, 0.0053, 0.0053, 0.0053, 0.0053, 0.0053]
t: 3501 | D: -0.004125669242727836 M: 0.0169 S: 0.0053
3502	86    	[  21.1         484.27833043]	[  14.78952332  202.01190607]	[ 0.  0.]                  	[  50.          754.83691157]
 Testing termination:
Matching M ->  0.0169
against ->  [0.017, 0.017, 0.017, 0.017, 0.017, 0.017, 0.017, 0.017, 0.017, 0.017, 0.017, 0.017, 0.0169, 0.0169, 0.0169, 0.0169, 0.0169, 0.0169, 0.0169, 0.0169, 0.0169, 0.0169, 0.0169, 0.0169, 0.0169, 0.0169, 0.0169, 0.0169, 0.0169]
 Testing terminat

t: 3553 | D: -0.01330623925456293 M: 0.0169 S: 0.0052
3554	91    	[  22.18        501.84437428]	[  14.49370898  193.0569303 ]	[ 0.  0.]                  	[  50.          754.83691157]
 Testing termination:
Matching M ->  0.0169
against ->  [0.0169, 0.0169, 0.0169, 0.0169, 0.0169, 0.0169, 0.0169, 0.0169, 0.0169, 0.0169, 0.0169, 0.0169, 0.0169, 0.0169, 0.0169, 0.0169, 0.0168, 0.0168, 0.0168, 0.0169, 0.0168, 0.0168, 0.0169, 0.0169, 0.0169, 0.0169, 0.0169, 0.0169, 0.0169]
 Testing termination:
Matching S ->  0.0052
against ->  [0.0053, 0.0053, 0.0053, 0.0053, 0.0053, 0.0053, 0.0053, 0.0053, 0.0053, 0.0053, 0.0053, 0.0052, 0.0052, 0.0052, 0.0052, 0.0052, 0.0052, 0.0052, 0.0052, 0.0052, 0.0052, 0.0052, 0.0052, 0.0053, 0.0053, 0.0053, 0.0053, 0.0053, 0.0052]
t: 3554 | D: 0.057325108651275006 M: 0.0169 S: 0.0052
3555	93    	[  20.76        483.72554871]	[  14.14716933  196.76128953]	[ 0.  0.]                  	[  50.          754.83691157]
 Testing termination:
Matching M ->  0.0169
against ->

against ->  [0.0052, 0.0052, 0.0052, 0.0052, 0.0052, 0.0052, 0.0052, 0.0052, 0.0052, 0.0052, 0.0052, 0.0052, 0.0052, 0.0052, 0.0052, 0.0052, 0.0052, 0.0052, 0.0052, 0.0052, 0.0052, 0.0052, 0.0052, 0.0052, 0.0052, 0.0052, 0.0052, 0.0052, 0.0052]
t: 3606 | D: 0.00010346422684624768 M: 0.0168 S: 0.0052
3607	89    	[  20.96        474.77229744]	[  15.14590374  216.43489114]	[ 0.  0.]                  	[  50.          754.83691157]
 Testing termination:
Matching M ->  0.0168
against ->  [0.0169, 0.0169, 0.0169, 0.0169, 0.0169, 0.017, 0.017, 0.017, 0.017, 0.0169, 0.0169, 0.0169, 0.0169, 0.0169, 0.0169, 0.0169, 0.0169, 0.0169, 0.0169, 0.0169, 0.0169, 0.0169, 0.0169, 0.0169, 0.0169, 0.0169, 0.0168, 0.0168, 0.0168]
 Testing termination:
Matching S ->  0.0052
against ->  [0.0052, 0.0052, 0.0052, 0.0052, 0.0052, 0.0052, 0.0052, 0.0052, 0.0052, 0.0052, 0.0052, 0.0052, 0.0052, 0.0052, 0.0052, 0.0052, 0.0052, 0.0052, 0.0052, 0.0052, 0.0052, 0.0052, 0.0052, 0.0052, 0.0052, 0.0052, 0.0052, 0.0052, 0.0

t: 3658 | D: -0.0028655355907198365 M: 0.0167 S: 0.0052
3659	87    	[  23.12        508.10544447]	[  15.46045277  200.95887749]	[ 0.  0.]                  	[  50.          754.83691157]
 Testing termination:
Matching M ->  0.0167
against ->  [0.0167, 0.0167, 0.0167, 0.0167, 0.0167, 0.0167, 0.0167, 0.0166, 0.0166, 0.0166, 0.0166, 0.0166, 0.0166, 0.0166, 0.0166, 0.0167, 0.0166, 0.0167, 0.0168, 0.0168, 0.0168, 0.0167, 0.0167, 0.0167, 0.0167, 0.0167, 0.0167, 0.0167, 0.0167]
 Testing termination:
Matching S ->  0.0052
against ->  [0.0052, 0.0052, 0.0052, 0.0052, 0.0052, 0.0052, 0.0052, 0.0052, 0.0052, 0.0052, 0.0052, 0.0052, 0.0052, 0.0051, 0.0051, 0.0051, 0.0051, 0.0052, 0.0052, 0.0052, 0.0052, 0.0052, 0.0052, 0.0052, 0.0052, 0.0052, 0.0052, 0.0052, 0.0052]
t: 3659 | D: -0.0019231815564772423 M: 0.0167 S: 0.0052
3660	89    	[  23.16        505.05786671]	[  15.77004756  208.92104709]	[ 0.  0.]                  	[  50.          754.83691157]
 Testing termination:
Matching M ->  0.0167
agains

against ->  [0.0051, 0.0051, 0.0051, 0.0051, 0.0051, 0.0051, 0.0051, 0.0051, 0.0051, 0.0051, 0.0051, 0.0051, 0.0051, 0.0051, 0.0051, 0.0051, 0.0051, 0.0051, 0.0051, 0.0051, 0.0051, 0.0051, 0.0051, 0.0051, 0.0051, 0.0051, 0.0051, 0.0051, 0.0051]
t: 3711 | D: -0.0006018448918279108 M: 0.0166 S: 0.0051
3712	86    	[  19.72        467.14810668]	[  14.23803357  200.72421107]	[ 0.  0.]                  	[  50.          754.83691157]
 Testing termination:
Matching M ->  0.0166
against ->  [0.0167, 0.0167, 0.0167, 0.0167, 0.0167, 0.0168, 0.0167, 0.0167, 0.0167, 0.0167, 0.0167, 0.0167, 0.0167, 0.0167, 0.0167, 0.0167, 0.0167, 0.0167, 0.0167, 0.0167, 0.0167, 0.0167, 0.0167, 0.0167, 0.0167, 0.0167, 0.0167, 0.0166, 0.0166]
 Testing termination:
Matching S ->  0.0051
against ->  [0.0051, 0.0051, 0.0051, 0.0051, 0.0051, 0.0051, 0.0051, 0.0051, 0.0051, 0.0051, 0.0051, 0.0051, 0.0051, 0.0051, 0.0051, 0.0051, 0.0051, 0.0051, 0.0051, 0.0051, 0.0051, 0.0051, 0.0051, 0.0051, 0.0051, 0.0051, 0.0051, 0.0051,